In [1]:
# API = Application Programming Interface
# API is a thing that allows two software apps to communicate 
# How? with standard ways of doing things to they speak the same language

#note you need to understand how to work with command line

In [ ]:
!pip install flask

# Toy Example

In [15]:
import threading
import subprocess
import time

def run_server():
    # Use subprocess.Popen to run the server.py script non-blockingly
    # and redirect output to stdout (so you can see it in the Jupyter notebook)
    server_process = subprocess.Popen(["python", "assets/server.py"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Print server output in real-time
    for line in server_process.stdout:
        print(line, end='')

    # Wait for the server process to terminate
    server_process.wait()

# Function to stop the server gracefully
def stop_server(server_thread, server_process):
    if server_process:
        server_process.terminate()  # or .kill() if .terminate() is not effective
    server_thread.join()

# Start the server as a background daemon thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait a bit for the server to start
time.sleep(3)  # Adjust this sleep time as needed
print("Server should be running now.")

# Example usage to stop the server, showing the principle (you need the server_process for this to work):
# stop_server(server_thread, server_process)


 * Serving Flask app 'server'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 105-378-404
Server should be running now.


In [16]:
import requests

base_url = "http://127.0.0.1:5000/weather/"
city = "New York"
response = requests.get(base_url + city)

if response.status_code == 200:
    data = response.json()
    # Assuming the city's weather data is nested under the city name
    city_weather = data.get(city, {})  # Use .get to avoid KeyError if city is not found
    temperature = city_weather.get('temperature', 'No temperature data')
    condition = city_weather.get('condition', 'No condition data')
    print(f"Weather in {city}: {temperature}, {condition}")
else:
    print("Failed to retrieve data.")


Weather in New York: 5°C, Cloudy


In [31]:
import requests

# Base URL of the Flask server
base_url = "http://127.0.0.1:5000/weather"

# CREATE: Add a new city's weather data
def create_weather(city, temperature, condition):
    response = requests.post(base_url, json={"city": city, "temperature": temperature, "condition": condition})
    if response.status_code == 201:
        print(f"Created weather data for {city}.")
    else:
        print(f"Failed to create data. Status code: {response.status_code}, Response: {response.text}")


# READ: Get weather data for a specific city
def read_weather(city):
    response = requests.get(base_url + city)
    if response.status_code == 200:
        data = response.json()
        city_weather = data.get(city, {})
        temperature = city_weather.get('temperature', 'No temperature data')
        condition = city_weather.get('condition', 'No condition data')
        print(f"Weather in {city}: {temperature}, {condition}")
    else:
        print(f"Failed to retrieve data for {city}.")

# UPDATE: Update a city's weather data
def update_weather(city, temperature=None, condition=None):
    update_data = {}
    if temperature:
        update_data['temperature'] = temperature
    if condition:
        update_data['condition'] = condition
    response = requests.put(base_url + city, json=update_data)
    if response.status_code == 200:
        print(f"Updated weather data for {city}.")
    else:
        print(f"Failed to update data for {city}.")

# DELETE: Delete a city's weather data
def delete_weather(city):
    response = requests.delete(base_url + city)
    if response.status_code == 200:
        print(f"Deleted weather data for {city}.")
    else:
        print(f"Failed to delete data for {city}.")

In [34]:
# Example usage of CRUD operations
create_weather("Boston", "10°C", "Rainy")

Created weather data for Boston.


In [29]:
read_weather("Boston")

Failed to retrieve data for Boston.


In [30]:
update_weather("Boston", temperature="15°C", condition="Sunny")

Failed to update data for Boston.


In [33]:
read_weather("Boston")

Failed to retrieve data for Boston.


In [32]:
delete_weather("Boston")

Failed to delete data for Boston.


# Real Life Example

In [2]:
import requests

# Function to get the office ID and grid points for a given latitude and longitude
def get_gridpoint(latitude, longitude):
    url = f"https://api.weather.gov/points/{latitude},{longitude}"
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError if the response was an error
    data = response.json()
    return data['properties']['gridId'], data['properties']['gridX'], data['properties']['gridY']

# Function to get the weather forecast using the office ID and grid points
def get_forecast(gridId, gridX, gridY):
    url = f"https://api.weather.gov/gridpoints/{gridId}/{gridX},{gridY}/forecast"
    headers = {'User-Agent': 'Mozilla/5.0'}  # Some APIs require a user-agent header
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data

# Main program
if __name__ == "__main__":
    # Coordinates for Central Park, New York City
    latitude = 40.78
    longitude = -73.97

    try:
        # Get office ID and grid points
        gridId, gridX, gridY = get_gridpoint(latitude, longitude)
        print(f"Grid Point Info: {gridId}, {gridX}, {gridY}")

        # Get forecast
        forecast_data = get_forecast(gridId, gridX, gridY)
        for period in forecast_data['properties']['periods'][:2]:  # Print the first 2 forecast periods
            print(f"\n{period['name']}: {period['detailedForecast']}")
    except requests.HTTPError as e:
        print("HTTP Error:", e)
    except Exception as e:
        print("Error:", e)


Grid Point Info: OKX, 34, 38

This Afternoon: A slight chance of rain showers after 5pm. Mostly sunny, with a high near 60. West wind around 13 mph, with gusts as high as 25 mph. Chance of precipitation is 20%.

Tonight: A chance of rain showers before 5am. Mostly cloudy. Low around 46, with temperatures rising to around 49 overnight. West wind 3 to 9 mph. Chance of precipitation is 50%. New rainfall amounts less than a tenth of an inch possible.
